# Recommendation

Approche choisie pour créer un système de recommandation : Algorithme de clustering

In [118]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import json

Récupère les données des images favorites de l'utilisateur

In [168]:
# Charge les données du fichier JSON
with open("profil.json", "r", encoding="utf-8") as file:
    json_data = json.load(file)

# Extrait les données et les résultats
images = json_data["images"]
data = json_data["data"]
result = json_data["resultats"]

# Crée le DataFrame
data_images = pd.DataFrame(images, columns=["nom_image"])
# Ajoute les colonnes en décomposant les données de "data"
data_expanded = pd.DataFrame(data, columns=["orientation", "qualite", "animal", "nature", "luminosite"])
# Et concatène les deux DataFrames
dataframe = pd.concat([data_images, data_expanded], axis=1)

given_images = dataframe
given_images = given_images.values.tolist()

# Ajoute la colonne des résultats
dataframe['result'] = result

# Filtre les données pour ne garder que les images favorites
df_favorite = dataframe[dataframe['result'] == 'Favorite']

# Supprime la colonne 'result' après le filtrage
df_favorite = df_favorite.drop(columns=["result"])

# Convertit le DataFrame en une liste
favorite_list = df_favorite.values.tolist()

In [171]:
print("longueur :", len(df_favorite))
print(favorite_list)
print(df_favorite)

longueur : 10
[['68.jpg', 'Paysage', 'basse', 'Chien', 'pas de nature', 'sombre'], ['34.jpg', 'Paysage', 'basse', 'Chien', 'en nature', 'sombre'], ['81.jpg', 'Paysage', 'basse', 'Chien', 'pas de nature', 'lumineux'], ['98.JPG', 'Paysage', 'moyenne', 'Chien', 'en nature', 'lumineux'], ['59.jpg', 'Portrait', 'haute', 'Chien', 'en nature', 'sombre'], ['75.jpg', 'Paysage', 'basse', 'Chien', 'en nature', 'sombre'], ['102.jpg', 'Paysage', 'haute', 'Chien', 'pas de nature', 'sombre'], ['44.jpg', 'Paysage', 'moyenne', 'Chien', 'pas de nature', 'sombre'], ['46.jpg', 'Paysage', 'moyenne', 'Chien', 'en nature', 'sombre'], ['105.jpg', 'Paysage', 'moyenne', 'Chien', 'pas de nature', 'sombre']]
   nom_image orientation  qualite animal         nature luminosite
1     68.jpg     Paysage    basse  Chien  pas de nature     sombre
3     34.jpg     Paysage    basse  Chien      en nature     sombre
7     81.jpg     Paysage    basse  Chien  pas de nature   lumineux
10    98.JPG     Paysage  moyenne  Chien  

Récupère les données de toutes les images existantes et enlève les 20 images proposées à l'utilisateur

In [ ]:
# Charge les données du fichier JSON
with open("profil.json", "r", encoding="utf-8") as file:
    json_data = json.load(file)

# Récupère la liste des chemins des images
img_paths = json_data["images"]

In [ ]:
# Charge les données du fichier JSON
with open("etiquetage.json", "r", encoding="utf-8") as file:
    json_data = json.load(file)

# Crée le DataFrame
dataframe = pd.DataFrame(json_data, columns=["nom_image", "orientation", "qualite", "animal", "nature", "luminosite"])

# Filtrer les données pour exclure les images spécifiées
dataframe_filtered = dataframe[~dataframe["nom_image"].isin(img_paths)]

# Convertit le DataFrame en une liste
data_list = dataframe.values.tolist()
data_list_filtered = dataframe_filtered.values.tolist()

In [158]:
print("longueur :", len(data_list))
print(data_list)
print(data_list_filtered)
print(dataframe_filtered)

longueur : 116
[['1.jpg', 'Paysage', 'moyenne', 'Chien', 'pas de nature', 'sombre'], ['10.jpg', 'Portrait', 'moyenne', 'Chat', 'pas de nature', 'lumineux'], ['100.jpg', 'Paysage', 'moyenne', 'Chien', 'pas de nature', 'lumineux'], ['101.png', 'Portrait', 'moyenne', 'Chat', 'pas de nature', 'sombre'], ['102.jpg', 'Paysage', 'haute', 'Chien', 'pas de nature', 'sombre'], ['103.jpg', 'Paysage', 'basse', 'Chien', 'pas de nature', 'lumineux'], ['104.jpg', 'Paysage', 'haute', 'Chien', 'en nature', 'sombre'], ['105.jpg', 'Paysage', 'moyenne', 'Chien', 'pas de nature', 'sombre'], ['106.jpg', 'Portrait', 'basse', 'Chat', 'pas de nature', 'sombre'], ['107.jpg', 'Portrait', 'moyenne', 'Chat', 'pas de nature', 'lumineux'], ['108.jpg', 'Paysage', 'moyenne', 'Chat', 'pas de nature', 'lumineux'], ['109.jpg', 'Paysage', 'moyenne', 'Chat', 'en nature', 'lumineux'], ['11.jpg', 'Paysage', 'moyenne', 'Chat', 'pas de nature', 'sombre'], ['110.jpg', 'Portrait', 'moyenne', 'Chat', 'pas de nature', 'lumineux'],

In [ ]:
# Encode categorical features
label_encoders = [LabelEncoder() for _ in range(len(data_list[0]))]
encoded_data = []
for i, column in enumerate(zip(*data_list)):
    encoded_data.append(label_encoders[i].fit_transform(column))

X = list(zip(*encoded_data))  # Features

# Clustering
k = 2  # Number of clusters
kmeans = KMeans(n_clusters=k, n_init=10)
kmeans.fit(X)
clusters = kmeans.labels_

# Add cluster labels to the original data
data_with_clusters = pd.DataFrame(data_list, columns=["nom_image", "orientation", "qualite", "animal", "nature", "luminosite"])
data_with_clusters["Cluster"] = clusters

# Recommendation function
def recommend_items(cluster, data_with_clusters, given_images):
    items_in_cluster = data_with_clusters[data_with_clusters["Cluster"] == cluster]
    # Exclude favorite items from recommendations
    items_in_cluster = items_in_cluster[~items_in_cluster["nom_image"].isin(given_images)]
    recommended_items = items_in_cluster.sample(n=4)  # Sample 4 items from the cluster
    return recommended_items

for img_fav in favorite_list:
    # Encode user interaction
    user_interaction = img_fav  # Assuming user interacted with this item
    encoded_interaction = [label_encoders[i].transform([val])[0] for i, val in enumerate(user_interaction)]
    cluster = kmeans.predict([encoded_interaction])[0]

# Get recommendations
recommendations = recommend_items(cluster, data_with_clusters, given_images)
print("Recommended items:")
print(recommendations)

Recommended items:
    nom_image orientation  qualite animal         nature luminosite  Cluster
102    87.jpg     Paysage    basse   Chat  pas de nature   lumineux        1
70     58.jpg     Paysage    basse  Chien  pas de nature     sombre        1
83      7.jpg       Carré  moyenne  Chien  pas de nature   lumineux        1
106    90.jpg     Paysage  moyenne  Chien  pas de nature     sombre        1
Recommended items:
   nom_image orientation  qualite animal         nature luminosite  Cluster
18   115.jpg    Portrait    haute   Chat  pas de nature     sombre        0
0      1.jpg     Paysage  moyenne  Chien  pas de nature     sombre        0
20    12.JPG     Paysage  moyenne   Chat  pas de nature     sombre        0
21    13.jpg       Carré    basse  Chien      en nature   lumineux        0
Recommended items:
    nom_image orientation  qualite animal         nature luminosite  Cluster
114    98.JPG     Paysage  moyenne  Chien      en nature   lumineux        1
71     59.jpg    Portrai